## Using Miltple Linear Regression models to predict depression levels based on daily patterns

### Imports

In [1]:
import modules
import numpy as np
from sklearn.linear_model import LinearRegression

In [2]:
#get data with averages taken over weekly increments
X = modules.get_and_avg_data(7)
Y = modules.categorize_depression(X['depression_class'])
X = X.drop(columns={'depression_score', 'depression_class'}, axis=1)

no user_tags: 501
no user_tags: 503
no user_tags: 507
no user_tags: 520
no user_tags: 523
no user_tags: 529
no user_tags: 532
no user_tags: 544
no user_tags: 547
no user_tags: 634
no user_tags: 658
no user_tags: 661
no user_tags: 662
no user_tags: 664
no user_tags: 668
